In [138]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [4]:
def retrieve_professions_from_field_first(field):
    h3_elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Professions')]")
    print(h3_elements)
    if len(h3_elements)==0:
        raise Exception(f"Aucun element trouvé pour la première fonction pour {field}")
    for h3_element in h3_elements:
        ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")
        driver.execute_script("arguments[0].scrollIntoView(true);", h3_element)
        if ul_element:
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    data[field].append(text)
        time.sleep(0.5)
    # driver.back()



def retrieve_professions_from_field_second(field):
    strong_elements = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Professions')]]")
    print(f"strong: {strong_elements}")
    if len(strong_elements)==0:
        errors.append(field)
    for strong_element in strong_elements:
        ul_element = strong_element.find_element(By.XPATH, "following-sibling::ul")
        if ul_element:
            driver.execute_script("arguments[0].scrollIntoView(true);", ul_element)
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    data[field].append(text)
        time.sleep(0.5)
    # driver.back()

def retrieve_professions_from_field(field):
    global nb_of_functions_tried, nb_of_functions
    try:
        if nb_of_functions_tried==1:
            retrieve_professions_from_field_first(field)
            driver.back()
        if nb_of_functions_tried==2:
            retrieve_professions_from_field_second(field)
            driver.back()
            nb_of_functions_tried=1
    except Exception as e:
        print(e)
        nb_of_functions_tried+=1
        retrieve_professions_from_field(field)

    if nb_of_functions_tried > nb_of_functions:
        driver.back()
        nb_of_functions_tried=1
    return



In [45]:
driver = webdriver.Chrome()
driver.get("https://www.ulaval.ca/etudes/programmes")
title = driver.title

# Keys represente fields and values are arrays whose contains professions 
data={}
errors=[]
nb_of_functions=2
nb_of_functions_tried=1

pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
print(pagination_next_element)
while pagination_next_element:
    results_element= driver.find_element(By.ID,"resultats")
    children=results_element.find_elements(By.XPATH, "./ul[@class='views-row']")

    for child in children:
        element=child.find_element(By.XPATH, "./li/a")
        field=element.find_element(By.XPATH,"./div[@class='main-info']/h3[@class='titre-programme']").text
        data[field]=[]
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        retrieve_professions_from_field(field)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", pagination_next_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", pagination_next_element.find_element(By.TAG_NAME, "a"))
    pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")


print(data)


<selenium.webdriver.remote.webelement.WebElement (session="2fdbcd4890567ad24f147fc55e41edec", element="f.CB0AAFA66B4BB810B308B0CAB99173B3.d.211A26BED07FBF8F5D2E1D7E53C1B6EA.e.81")>


NameError: name 'retrieve_professions_from_field' is not defined

In [139]:
driver = webdriver.Chrome()
def indeedDataFill(ulavalPosition, location):
    
    driver.get("https://emplois.ca.indeed.com/")
    title_input = driver.find_element(By.XPATH, "//input[@id='text-input-what']")
    location_input = driver.find_element(By.XPATH, "//input[@id='text-input-where']")
    location_input.clear()
    location_input.send_keys(location)
    title_input.send_keys(ulavalPosition, Keys.RETURN)

    return title_input, location_input
    

In [140]:
indeedDataFill("Actuaire", "Canada")

(<selenium.webdriver.remote.webelement.WebElement (session="62d318b10e263a13116baa1dec97d59c", element="f.46F099D944433B1C11F3C4FBF44E6FDA.d.00C4C06E6A1CF1E46C2E20D45799EB58.e.1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="62d318b10e263a13116baa1dec97d59c", element="f.46F099D944433B1C11F3C4FBF44E6FDA.d.00C4C06E6A1CF1E46C2E20D45799EB58.e.3")>)

In [150]:
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

def stateDateToDate(stateDate):
   date=None
   for str in stateDate.split(" "):
      try:
         date= datetime.strptime(str, '%Y-%m-%d')
      except ValueError:
         continue
      if bool(re.compile(r'\b(\w*\+\w*)\b', re.IGNORECASE).search(str)):
         str= str.replace("+", "")
      if str.isnumeric():
            if bool(re.compile(r'\b(\w*jour\w*|\w*day\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(days=int(str)))
            if bool(re.compile(r'\b(\w*mois\w*|\w*month\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(months=int(str)))
            if bool(re.compile(r'\b(\w*an \w*|\w*year\w*|\w*année\w*)\b', re.IGNORECASE).search(stateDate)):
               date= time.strftime('%Y-%m-%d', datetime.now()-relativedelta(years=int(str)))
            
   return date

In [151]:
def indeedPositionInfos():
    positionsDescriptionList = driver.find_element(By.XPATH, "//div[@id='mosaic-provider-jobcards']")
    keys = ["positionTitle", "companyName", "location", "description", "datePosted", "salary", "jobType"]
    positionsDetails = {}
    ulElement = positionsDescriptionList.find_element(By.XPATH, "./ul")
    liElements = ulElement.find_elements(By.XPATH, "./li[@class='css-5lfssm eu4oa1w0']")
    for li in liElements:
        driver.execute_script("arguments[0].scrollIntoView(true);", li)
        time.sleep(0.5)
        try:
          position=li.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]/a")
          positionTitle = driver.execute_script("return arguments[0].innerText;", position)
        except Exception as e:
            print("No position found")
            position=None

        try:
          dateElement=li.find_element(By.XPATH, ".//span[@data-testid, 'myJobsStateDate']")
          positionPostedDate=stateDateToDate(driver.execute_script("return arguments[0].innerText;", dateElement))
        except Exception as e:
            print("No position found")
            dateElement=None
        
        if position :
            driver.execute_script("arguments[0].click();", position)
            time.sleep(0.5)
            try:
               positionsDetails = driver.find_element(By.XPATH, "//div[@id='jobsearch-ViewjobPaneWrapper']")
            except Exception as e:
               positionsDetails=None
               # raise e  #print("No position details found")
               
            
            if positionsDetails:	
                try:
                    companyName = driver.execute_script("return arguments[0].innerText;",positionsDetails.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyName']"))
                except Exception as e:
                    print("No company name found")
                    companyName=None
                try:
                    location = driver.execute_script("return arguments[0].innerText;",positionsDetails.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyLocation/div']"))
                except Exception as e:  
                    print("No location found")
                    location=None
                try:
                    jobType = driver.execute_script("return arguments[0].innerText;",positionsDetails.find_element(By.XPATH, ".//div[@id='salaryInfoAndJobType/span']"))
                except Exception as e:
                    print("No job type found")
                    jobType=None
                try:
                    description = positionsDetails.find_element(By.XPATH, ".//div[@id='jobDescriptionText/div']"))
                except Exception as e:
                    print("No description found")
                    description=None


       

In [149]:
try:
    pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li[@data-testid='pagination-page-next']/a")
except Exception as e:
    pagination_next_element=None

print(pagination_next_element)
if not pagination_next_element:
    indeedPositionInfos()
if pagination_next_element:
    while pagination_next_element:
        indeedPositionInfos()
        try:
            pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li[@data-testid='pagination-page-next']/a")
        except Exception as e:
            pagination_next_element=None


None

 position title: Administrateur de régime de retraite

 position title: Analyste en assurance collective

 position title: Opportunités de carrière en actuariat

 position title: CONSEILLER(-ÈRE) - ADMINISTRATION DES RÉGIMES DE RETRAITE

 position title: Directeur(rice) actuaire en assurance vie
No position found

 position title: Directeur de comptes - Institutionnel et gestion privée

 position title: Conseiller⸱ère adjoint⸱e – Retraite

 position title: Conseiller(ère) principal(e) en actuariat corporatif

 position title: Analyst, corporate actuarial – Canadian Society

 position title: Spécialiste en comptabilité
No position found

 position title: Analyste actuariel principal/ chef d'équipe

 position title: Agente ou agent de probation

 position title: Directeur ou directrice en assurance collective

 position title: Scientifique de données - Quantification des risques

 position title: Analyste d'affaires principal(e) – IA & Télématique
No position found

 position title